In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from missingpy import KNNImputer
import copy
from sklearn.preprocessing import LabelEncoder;


In [6]:
df=pd.read_csv('train.csv')

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# 80% of counts are 

class columnTypeIdentification:
    def __init__(self, df):
        self.dtypes={}
        self.df=df
        for i in self.df.columns:
            self.dtypes[i]=(self.df[i].dtypes)
#         print(self.dtypes)
        self.colTypes={'Categorical': [], 'Text':[], 'Numeric': []}
        self.detectingColTypes()
        target_type=''
        
        for i in self.colTypes.keys():
            if y in self.colTypes[i]:
                target_type=i
                break
        self.target_type=target_type
    
    def detectingColTypes(self):
        for i in self.dtypes.keys():
            if self.dtypes[i]=='O':
                if (df[i].fillna('',axis=0).apply(lambda x: len(x))).quantile(q=0.95)<20:
                    self.colTypes['Categorical'].append(i)
                else:
                    self.colTypes['Text'].append(i)
            else: 
                distinctValues = self.df[i].nunique()
                if distinctValues < int((self.df[i].shape[0])*0.05):
                    self.colTypes['Categorical'].append(i)
                else:
                    self.colTypes['Numeric'].append(i)
    

In [9]:
class nullHandling():
  
    def __init__(self, df):
        self.dict_isnull = (df.isnull().sum() / len(df)).to_dict()
        self.df=df
        self.colTypes=colTypes.copy()
 
    
    def remove_columns(self):
        cols_remove=[]
        for key in self.dict_isnull:
            if(self.dict_isnull[key]>0.75):
                cols_remove.append(key)        
        if not cols_remove:
            return self.colTypes
        else:
            for i in cols_remove:
                for j in self.colTypes.keys():
                    if i in self.colTypes[j]:
                        self.colTypes[j].remove(i)
        if y in cols_remove:
            cols_remove.remove(y)
        
        self.df.drop(cols_remove,axis=1, inplace=True)
        
#         return self.colTypes
        
    
    def continuous_impute_zero(self):
        df_temp=self.df.copy()
        df_temp[self.colTypes['Numeric']]=df_temp[self.colTypes['Numeric']].fillna(0)
        return df_temp
    
    def continuous_impute_mean(self):
        df_temp=self.df.copy()
        imputer = SimpleImputer(strategy='mean')
        df_temp[self.colTypes['Numeric']] = imputer.fit_transform(df_temp[self.colTypes['Numeric']])
        return df_temp
    
    def continuous_impute_knn(self):
        df_temp=self.df.copy()
        imputer = KNNImputer(n_neighbors=5) 
        df_temp[self.colTypes['Numeric']] = imputer.fit_transform(df_temp[self.colTypes['Numeric']])
        return df_temp
    
        
    def impute(self,strategy,fill_value = 0, fill_categorical = '-1'):
        df_temp=self.df
        
        #Dealing wit Continuous cols
        if strategy is None:
            df_temp[self.colTypes['Numeric']]=df_temp[self.colTypes['Numeric']].fillna(fill_value)
        elif strategy == 'mean':
            self.continuous_impute_mean()
        elif strategy == 'knn':
            self.continuous_impute_knn()
            
        #dealing with categorical Colsdata:image/pjpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQH/2wBDAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQH/wAARCABAAEADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD+3iiiiv5XP1AKKKKACiiigAooooAKKK+P/wBvvS/irrX7Hnx40z4K/wBsN8RLrwfGumQ+HjMNeu9Hj1vSZfGVjowtv9Ll1O/8Fx+ILOzt7INf3U06W1ipvJYBXPi67wuExWKVKpXeGw9auqFJXqVnRpyqKlTVnedTl5YqzvJrR7GlGn7WtSpOcaaq1IU/aT0jDnko88v7sb3fkmdxr37Xf7K/hfXpvC/iL9o34I6L4gtbhrS80nUvif4NtLywu0Yo9rqMcusL/Z9yjja8F6YJUbAZBmvUfFHxM+HHgjw5ZeMfGnxA8E+EPCOpyWUWm+KfE/irQtA8OahLqVtJeadFZa3qt/aaZdSahZwzXVkkF1I11bRSTwCSJGYfhr/wTu8Af8ExPjj8E/Bfwy1LwB8OtU+Pkfh9LT4keHfiLDcW3xG1bxVBE41/U/Cuo39zb3V1pEsqTXenQeDL1ZdE03yItStbG+hnZvXv+CzXh3RvCn7DPw98JaLZm28PeGvi78LfDmk2D3FzcG30bRvCPjHTLC0a6uJZbucxWNtFCbieeS5kKmSWV5WZz8vTz/H/ANg4zPZwymvRp4NYrD0sDiq9Zxq2Up4XFVJ0YqM6alGM3BRnGalGVJaM9V5dQ+v0MvTxlOpOv7KpKvRhTvC9o1aMVOTcZtScbtpx5Wps/Ynw74k8O+L9E07xL4S17RfFHhzV4PtOk6/4d1Sx1rRdTtt7xfaNP1XTZ7mxvYPMjkj822nkj3o6btykDnPHfxV+F/wuh064+JnxH8B/Du31iW5h0ifx14v8P+EodUms0ie7i06XX9Q09L2W1S4ge4jtmleBJomkCiRCfzC/4Ja+INY+FmoftE/sL+NbyWXxF+zb8RtU1PwO92Skur/Cvxpeyalpt/ZQn5vsovriHxFKxCiNPHWnwgfIcfmV/wAFWvEmqftFfEv49eK9OuZJfhb+xtZeBPhDpcsTsbO/+LnxE8V21x4ykhyQrS2Flo+p+HdYRd3kXHhrR3ZFF1HI85hxVLCcPUc2p4WM8fVdWk8BOo0qWJwUa880pznGLkoYSGDxbUuVOTjSTS9omVh8pVbMpYOVaUcPFQqLEKKblSxDprCyUb8t60q9FWUnZOT15Wf1T2l3a39rbX1jc297ZXtvDd2d5aTR3Nrd2tzGs1vc21xCzwz288LpLDNE7xyxurozKwJsV5z8Hv8Akknwt/7Jz4I/9RnTK9Gr6+lP2lOnO1ueEJ23tzRTt8rnjSXLKUd+WTV/R2CvNPi/8XvAPwJ+H+s/FH4n6y/h7wP4eudBttb1pNP1DUxpo8ReIdL8M2FzNZaXbXl/JbLqesWQuntrWd4LYy3BjZImr0usXxF4b8O+L9F1Dw34s0DRfFHh3VoRb6poPiLS7HWtF1KBZEmWG/0vUoLmxvIlljjlWO4gkQSRo4G5FIVX2rpVVQdONd05+xlVjKdKNXlfs5VYRlCU6alZzjGcJSjdKSeoQ5OePtFJ0+aPOoNRk4XXMoycZJStezcZJOzcWtD+fD/gpdr3/BOfx/8ACbWvjV8K/iR8Nk/aettR0HV/Aeu/BrxBBH4z8Qa5/bmnNct4w0rw3IpSS3037XfjxH4ht9N1/TLuwsxZaq0m3Sr/AKT/AIKK618RfEX/AASr/Zs134tw3kHxI1bxZ8FL7xeNSga11SXVp/BPjB3u9WtGSN7TV76Ew3urWrxQvbajPcwPBA0ZiT9dPC/7HX7KXgrxDB4r8Kfs6/BvQvEdpcJd2Gr2Hw98NRXemXcbB47rSWOnMmk3ETANFNpqWskR/wBWy5NeveOvhx8Pviho0Xhz4leBvCHxB8PwX9vqsOh+NvDej+KdIi1S0jnhtdSi03W7O+s0v7aK6uYoLtYRcQx3E6RyKssgb4ypwxjcVHPKletl2Er5vl8ME6OXUK0MJKrCpOp9exftJc9bFPn9kpRhBxpXjKVVtSXuQzXD0ZZeqcMTWhgsU8Rz4mpB1VBxjH6vR5VywpLl52m3eeqjFaP8b/8Ago5q/iL9jf8AaN+EP7evgLSG1G01/wAIeLPgZ8UdJiHlWmsajP4e1TU/h1caqysplWW9to57mVmTZF4F0e3QSPKIz83fHz4Jal8GP+CN8Vx4rE83xJ+LvxI8EfGr4l6hfDOp3fib4ga5BqVtDqLOBMt7p3h5dFsdQhkYhNWh1KVVQzuK/on8a+AfA3xK0KTwv8RPBnhXx54amubW9l8P+MfD+leJtElvLKTzbO6k0vWbS9sXuLSX95bTNAZIJPnjZW5pvjX4feA/iT4fk8J/EPwX4U8d+Fpp7W6l8N+MPD2k+JNClubF/Msp5NJ1i0vLB5rSQB7aRoC8D/NEVPNa43hV4mrndSniowhmWBxVHCUJwk6eDxuYUaNHH4p2d37dYTDStHWMp4q1vau+dDNlShgIyotywmIozrVIy96vQw9SdTD0uy9n7arHXRpUr/AjI+D3/JJPhb/2TnwR/wCozplejVWsrKz02ztNO0+1t7GwsLaCysbKzgjtrSzs7WJILa1tbeFUht7e3hjSGCGJEjiiRURVVQBZr6+lF06dODd3CEINrryxSv8AOx48nzSlL+aTf3u4UUUVZIUUUUAFFFFABRRRQB//2Q==
        df_temp[self.colTypes['Categorical']]=df_temp[self.colTypes['Categorical']].fillna(fill_categorical)
        
        return df_temp


In [10]:
class FeatureReduction:

    def __init__(self,df):
        self.df=df
        self.colTypes=copy.deepcopy(colTypes)

        self.colTypes[target_type].remove(y)
        self.all_dfs=[]
        self.pearsons_corr()
#removing the continuous column with a correlation of above 0.8
    def pearsonCorr(self):
        self.colTypes['Numeric']=set(self.colTypes['Numeric']).intersection(set(self.df.columns))
        corr=self.df[list(self.colTypes['Numeric'])].corr(method="pearson").abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
        self.df.drop(to_drop, axis=1,inplace=True)
        self.colTypes['Numeric'] = [x for x in self.colTypes['Numeric'] if x not in to_drop]
        self.all_dfs.append(self.df)

        #returns the dict of dataframes updated in this class
    def returnDfs(self):
        return self.all_dfs
        


In [11]:
class Outlierhandling():
    def __init__(self,df):
        self.df=df.copy() 
        self.colTypes=copy.deepcopy(colTypes)
        
        self.colTypes[target_type].remove(y)
        
        self.colTypes['Numeric']= set(colTypes['Numeric']).intersection(set(df.columns))
        
        self.all_dfs=[]
        self.capping_outlier() 
     #   self.remove_outlier()
        self.zscore_outlier()
      
    def capping_outlier(self,lowerperc = 0.01,higherperc = 0.99):
        df = self.df[list(self.colTypes['Numeric'])].copy()
        df_out = self.df
        for col in df.columns:
            percentiles = df[col].quantile([lowerperc,higherperc]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
            df_out[list(self.colTypes['Numeric'])] = df
        self.all_dfs.append(df_out)
    
    
    def remove_outlier(self):
        df_out=self.df[list(self.colTypes['Numeric'])].copy()
        df_in = self.df.copy(deep=True)
        df_in.drop(list(self.colTypes['Numeric']),axis=1,inplace=True)
        for col_name in df_out.columns:
            q1 = df_out[col_name].quantile(0.25)
            q3 = df_out[col_name].quantile(0.75)
            iqr = q3-q1
            lower = q1-1.5*iqr
            upper = q3+1.5*iqr
            df_out = df_out.loc[(df_out[col_name] > lower) & (df_out[col_name] < upper)]
            df_final = pd.concat([df_in,df_out],axis = 1,join='inner')           
        self.all_dfs.append(df_final)
    
    
    def zscore_outlier(self,threshold = 3):
        l = []
        df1=self.df.copy(deep=True)
        df = self.df[list(self.colTypes['Numeric'])].copy(deep=True)
        df1.drop(self.colTypes['Numeric'],axis=1,inplace=True)
        df_out = pd.DataFrame()
        for i in df.columns:
            temp = []
            mean_1 = np.mean(df[i])
            std_1 =np.std(df[i])  
            for y in df[i]:
                z_score= (y - mean_1)/std_1 
                if np.abs(z_score) < threshold:
                    temp.append(y)
            df_temp = pd.DataFrame(temp)
            l.append(df_temp)
        df_out = pd.concat(l,axis = 1,join = 'inner')
        df_out.columns = df.columns
        df_final = pd.concat([df1,df_out],axis = 1,join='inner')  
        self.all_dfs.append(df_final)
    
    
    
    def return_dfs(self):
        return self.all_dfs

In [12]:
class Encoding:

    def __init__(self,df):
        self.colTypes=copy.deepcopy(colTypes)
#fetching column types for the columns in the current df
        self.colTypes['Categorical'] = set(df.columns).intersection(set(colTypes['Categorical']))
#removing the target column for the list of categorical columns
        if y in self.colTypes['Categorical']:
            self.colTypes['Categorical'].remove(y)
            
        self.df=df.copy()
        self.all_dfs = []
        self.oneHotEncode()
        self.labelEncode()
        
#encoding the categorical columns exclusing the target column
    def oneHotEncode(self):
        df1=self.df.copy(deep=True)
        df_y=pd.DataFrame()
        df1=pd.get_dummies(df1,drop_first=True,columns=list(self.colTypes['Categorical']))
        self.all_dfs.append(self.targetEncode(df1))

#encoding the categorical columns exclusing the target column
    def labelEncode(self):
        df1=self.df.copy(deep=True)
        df_y=pd.DataFrame()
        for x in self.colTypes['Categorical']:
            df1[x] = LabelEncoder.fit_transform(df1, y=df1[x])
        self.all_dfs.append(self.targetEncode(df1))

#encoding the categorical target column
    def targetEncode(self,t_df):
        if y not in self.colTypes['Numeric']:
             t_df[y] = LabelEncoder.fit_transform(t_df, y=t_df[y])
        return t_df
       
#returns the dict of dataframes updated in this class
    def returnDfs(self):
        return self.all_dfs


In [13]:
y='Survived'
#Col Indentification
colIdentObj=columnTypeIdentification(df)
colTypes=colIdentObj.colTypes
target_type=colIdentObj.target_type


#Null handling
nullHndlngObj=nullHandling(df)


# colTypes=nullHndlngObj.remove_columns()
df_dict={}
for strategy in [None, 'mean', 'knn']:
    df_dict['null_strategy_'+str(strategy)] = nullHndlngObj.impute(strategy)


#Feature Reduction
df_all=[]
for i in df_dict.keys():
    fRdctionObj=FeatureReduction(df_dict[i])
    df_all.extend(fRdctionObj.return_dfs())


#Outlier handling
df_all_oh=[]
for i in df_all:
    OH=Outlierhandling(i)
    df_all_oh.extend(OH.return_dfs())
    
    
#Encoding columns    
df_all_en=[]
for i in df_all_oh:
    en=Encoding(i)
    df_all_en.extend(en.return_dfs())    

    
    
#output of EDA : df_all_en


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\

In [14]:
colTypes['Numeric']

['PassengerId', 'Age', 'Fare']